<a href="https://colab.research.google.com/github/elkhayyat17/chat-with-pdfs-mistral-7b/blob/main/chat_with_your_pdfs_%5BRAG%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 163.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 243.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.55-cp310-cp310-manylinux_2_35_x86_64.whl size=26101061 sha256=381769eb93401f974bad1ce311573332ada73be5b39fb68afe517f78899ff68a
  Stored in directory: /tmp/pip-ephem-wheel-cache-yhev7y6b/wheels/35/c9/bb/019dbfeef119ab5c29f76574b76070afa7b7755ccfbb3ee226
Successfully built llama-cpp-python


In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Enable text wrapping so we don't have to scroll horizontally and create a function to flush CUDA cache.

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate


download document


In [ ]:
#!wget url of pdf
#or upload  pdf to content/Data
!wget https://solar-center.stanford.edu/teachers/Our-Solar-System-Book.pdf

--2024-03-03 06:00:06--  https://solar-center.stanford.edu/teachers/Our-Solar-System-Book.pdf
Resolving solar-center.stanford.edu (solar-center.stanford.edu)... 171.64.103.240
Connecting to solar-center.stanford.edu (solar-center.stanford.edu)|171.64.103.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5123417 (4.9M) [application/pdf]
Saving to: ‘Our-Solar-System-Book.pdf’

Our-Solar-System-Bo 100%[===================>]   4.89M  4.63MB/s    in 1.1s    

2024-03-03 06:00:09 (4.63 MB/s) - ‘Our-Solar-System-Book.pdf’ saved [5123417/5123417]



In [ ]:
!mkdir Data
!mv "Our-Solar-System-Book.pdf" "/content/Data"



In [ ]:
!pip install -U llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 45.2 MB/s eta 0:00:00


In [ ]:

documents = SimpleDirectoryReader("/content/Data/").load_data()

In [ ]:
%pip install llama-index-llms-llama-cpp


In [ ]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:38, 106.89it/s]                          
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large")
#HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")



config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:

service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-12-7e47785a9da0>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()


In [ ]:
response =  query_engine.query("summarize this book")
print(response)


llama_print_timings:        load time =     637.33 ms
llama_print_timings:      sample time =      72.68 ms /   144 runs   (    0.50 ms per token,  1981.34 tokens per second)
llama_print_timings: prompt eval time =     636.79 ms /   408 tokens (    1.56 ms per token,   640.72 tokens per second)
llama_print_timings:        eval time =    3508.20 ms /   143 runs   (   24.53 ms per token,    40.76 tokens per second)
llama_print_timings:       total time =    4605.98 ms /   551 tokens


 This book appears to be about the exploration and study of Mars, the fourth planet from the Sun in our solar system. The book discusses the history of Mars exploration, including early speculations about the famous "irrigation canals" on Mars, which were later debunked by images returned from NASA's Mariner 4 mission in 1965. The book also mentions NASA's Viking mission in the 1970s, which carried out life-detection experiments on the surface of Mars and suggested that the planet was lifeless. The book notes that the next two decades were met with struggle as several spacecraft from various countries were lost during Mars exploration.


In [ ]:
response =  query_engine.query("what do you know about moons of Uranus")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     596.91 ms
llama_print_timings:      sample time =      60.54 ms /   116 runs   (    0.52 ms per token,  1915.96 tokens per second)
llama_print_timings: prompt eval time =     934.05 ms /   641 tokens (    1.46 ms per token,   686.26 tokens per second)
llama_print_timings:        eval time =    3490.49 ms /   115 runs   (   30.35 ms per token,    32.95 tokens per second)
llama_print_timings:       total time =    4827.35 ms /   756 tokens


 Uranus has nine major, though extremely faint, rings and 27 known moons. Some of its moons include Miranda, Ariel, Umbriel, Titania, Oberon, and Neptune. These moons were discovered by William Herschel in the late 18th century and named after characters from William Shakespeare's plays. Uranus's moons are known for their unique characteristics such as their blue-green color, which is caused by the absorption of red light by methane gas in the upper atmosphere.


In [ ]:
import re
document_info = str(response.metadata)
resorce=str(re.findall(r"'page_label': '[^']*', 'file_name': '[^']*'", document_info))
print(resorce)

["'page_label': '37', 'file_name': '/content/Data/Our-Solar-System-Book.pdf'", "'page_label': '19', 'file_name': '/content/Data/Our-Solar-System-Book.pdf'"]


In [ ]:
global query_engine
def load_db(self):

    documents = SimpleDirectoryReader("/content/Data").load_data()
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    query_engine = index.as_query_engine()

    return query_engine


In [ ]:
import panel as pn
import param
import re

pn.extension()
class cbfs(param.Parameterized):
    answer = param.String("")
    resource =  param.List([])
    db_query  = param.String("")



    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.panels_ = []

        self.loaded_file = "Our-Solar-System-Book.pdf"
        self.qa = load_db(self.loaded_file)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
          file_input.save('/content/Data/'+file_input.filename)  # local copy
          self.loaded_file = file_input.filename
          button_load.button_style="outline"
          self.qa = load_db(self)
          button_load.button_style="solid"
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")



    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('**User:**', pn.pane.Markdown("", width=600)), scroll=True, styles={'background-color': '#FAF5E4'})
        response =  self.qa.query(query)
        ans=response.response
        document_info = str(response.metadata)
        self.db_query =str(re.findall(r"'page_label': '[^']*', 'file_name': '[^']*'", document_info))




        self.answer =ans

        self.panels.extend([
            pn.Row('**User:**', pn.pane.Markdown(query, width=600,styles={'background-color': '#FAF5E4'})),
            pn.Row('**ChatBot:**', pn.pane.Markdown(self.answer  , width=600, styles={'background-color': '#ffffff'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)


    @param.depends('db_query ', )
    def resource(self):
      if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#ffffff'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )


      self.panels_.extend([
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#ffffff'})),

            pn.pane.Str(self.db_query )
        ])
      return pn.WidgetBox(*self.panels_,scroll=True)





In [ ]:
cb = cbfs()


file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)

inp = pn.widgets.TextInput( placeholder='Massage your bot …', styles={'background-color': '#ffffff'},width=500)

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(

    pn.panel(conversation,  loading_indicator=True, height_policy="fit", styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),
    pn.Row(inp, styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),
)

tab2 = pn.Column(

    pn.panel(cb.resource, styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),

)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Chat With Your PDFs_Bot'), styles={'background-color': '#ffffff'},width=500),
    pn.Tabs(('Conversation', tab1), ('Resources', tab2),('Configure', tab4), styles={'background-color': '#ffffff'})
)
dashboard

Column
    [0] Row(styles={'background-color': '...}, width=500)
        [0] Markdown(str)
    [1] Tabs(styles={'background-color': '...})
        [0] Column
            [0] ParamFunction(function, _pane=WidgetBox, defer_load=False, height_policy='fit', loading_indicator=True, styles={'background-color': '...})
            [1] Divider()
            [2] Row(styles={'background-color': '...})
                [0] TextInput(placeholder='Massage your bot …', styles={'background-color': '...}, width=500)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False, styles={'background-color': '...})
            [1] Divider()
        [2] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Divider()

Llama.generate: prefix-match hit

llama_print_timings:        load time =     637.33 ms
llama_print_timings:      sample time =      78.28 ms /   140 runs   (    0.56 ms per token,  1788.43 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    4815.69 ms /   140 runs   (   34.40 ms per token,    29.07 tokens per second)
llama_print_timings:       total time =    5364.24 ms /   141 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     637.33 ms
llama_print_timings:      sample time =     124.61 ms /   218 runs   (    0.57 ms per token,  1749.50 tokens per second)
llama_print_timings: prompt eval time =     273.13 ms /    72 tokens (    3.79 ms per token,   263.61 tokens per second)
llama_print_timings:        eval time =    7809.28 ms /   217 runs   (   35.99 ms per token,    27.79 tokens per second)
llama_print_timings:       to